---

# TP 5 : Optimization  without constraints

<b>Sorbonne University</b><br>
Author : Marie Postel
---
$
\renewcommand{\R}{\mathbb{R}}
\newcommand{\eps}{\varepsilon}
$


In [ ]:
import numpy as np
from numpy import linalg
import matplotlib.pyplot as plt

<a id="dim2"></a>
## Part 1: Where Are My Keys?

We are interested in geolocating an object equipped with a GPS tag, whose position is pinpointed by a network of sensors that can transmit their distance to the tag.
The simplified problem is then formulated as follows: in the plane, three sources are placed, as well as an object whose position is unknown.
The GPS provides us with distances, tainted with errors, between each source and the object.
The objective is to obtain a good approximation of the object's position in the plane, knowing the positions of the sources, the distances, and the errors.

We denote $s_1$, $s_2$, and $s_3$ as the positions of the three sources in $\mathbb{R}^2$, and $x^\star \in \mathbb{R}^2$ as the position of the unknown object.
The distances measured by the sources are denoted $d_1$, $d_2$, and $d_3$.
It can be shown that $x^\star$ minimizes the following function, defined on $\mathbb{R}^2$,
$$ \tilde{J}(x) = -(\|x-s_1\| - d_1)^2+ (\|x-s_2\| - d_2)^2+ (\|x-s_3\| - d_3)^2. $$

#### Question 1
First, we consider dimension 1, and we study the function $f: \mathbb{R} \rightarrow \mathbb{R}$
$$f(x)=(|x-s| - d)^2$$
What problem do we encounter if we try to minimize $f$ using a method seen in class?

#### double click here then answer in text  or LaTeX or markdown format

#### Question 2
For $s=0$ and $d=1$, plot the graph of the function $f$ on the interval $[-2,2]$, as well as the graph of the function
$$f_\mu(x) = (\sqrt{(x-s)^2 + \mu} - d)^2$$ for $\mu=0.1$, $0.01$, and $0.001$.
We will discretize the study interval with 1000 points. There should be a single figure with 4 curves, labeled with a legend.

In [ ]:
# fill  here with your code
    

We return to our geolocation problem in the plane, which we now decide to approach by minimizing
J(x)=(∥x−s1∥2+μ−d1)2+(∥x−s2∥2+μ−d2)2+(∥x−s3∥2+μ−d3)2,J(x)=(∥x−s1​∥2+μ
​−d1​)2+(∥x−s2​∥2+μ
​−d2​)2+(∥x−s3​∥2+μ

​−d3​)2,
using a gradient descent method.

To simplify, we assume that all 3 sensors have the same margin of error.

Here, $x$, $s_1$, $s_2$, and $s_3 \in \mathbb{R}^2$, so $J: \mathbb{R}^2 \rightarrow \mathbb{R}$. Recall that $|v|^2=\sum_{i=1}^2v_i^2$.

Answer the following questions on the provided sheet: <br>
What is the derivative of $y\rightarrow \sqrt{y}$, for $y\in \mathbb{R}^{+*}$? <br>
For $x\in\mathbb{R}^2$, $s\in\mathbb{R}^2$, $d$, and $\mu\in\mathbb{R}$, compute the gradients of the following functions: <br>
$x\rightarrow \|x\|^2 $ <br>
$x\rightarrow \|x-s\|^2 $ with $x\in\mathbb{R}^2$ <br>
$x\rightarrow \sqrt{\|x-s\|^2+\mu} $ with $x\in\mathbb{R}^2$ <br>
$x\rightarrow (\sqrt{\|x-s\|^2+\mu}-d)^2 $ with $x\in\mathbb{R}^2$ <br>


#### double click here then answer in text  or LaTeX or markdown format

#### Question 3
In the following cell, define the function $J(x)$ and its gradient $gJ(x)$. This function should return a numpy array of the same dimensions as $x$. Do not modify the given values for $\mu$, $(s_i,d_i)_{i=1,2,3}$.

In [ ]:
mu=0.0001
s1=np.array([-1,1])
s2=np.array([2,1])
s3=np.array([1,-3])
d1,d2,d3=(1.4,2.3,3.)
d1,d2,d3=(1.41,2.23,3.16)

# fill  here with your code


Check your code by displaying the values of $J(x)$ and $gJ(x)$ for $x=\begin{pmatrix}1\\2\end{pmatrix}$.

Note: There is nothing to add in the following two cells, just execute them. If they give an error, correct your functions $J$ and/or $gJ$.

In [ ]:
#### Question 4.1
x=np.array([1,2])
print(J(x))
Jx=4.73351187568691
assert J(x)==Jx, "Your function J does not compute the right vakue"


In [ ]:
#### Question 4.2
print(gJ(x))
gJx=np.array([2.63135795, 3.26526551])
if not isinstance(gJ(x), np.ndarray):
    print("your gJ(x) function does not return a numpy array")
assert len(gJ(x))==2, "your gJ(x) function returns an array that does not have the same dimension as x"
assert np.linalg.norm(gJ(x)-gJx)<1.e-6 , "your gJ function does not return the correct values"


#### Question 5
In the following cell, minimize $J(x)$ using constant step size gradient descent.
Display the solution, the gradient of the function at this point, and the number of iterations.

In [ ]:
# fill  here with your code

#### Question 6
Display on a graph the three sources $s_1$, $s_2$, and $s_3$ with blue circles and the found target $x^*$ with a red cross.

In [ ]:
# fill  here with your code